In [19]:
from openai import OpenAI
from dotenv import load_dotenv
import os,re,json
load_dotenv()

import pandas as pd

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 1000)
pd.set_option("display.max_colwidth", 1000)
pd.set_option("display.max_info_columns", 1000)

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

### Creating Dataframe from json Data

In [20]:
df = pd.read_json("datas/raw_shopify_products.json")
df.shape

(2443, 92)

In [21]:
df.head(5)

,id,title,totalVariants,description,featuredImage,images,category,store_url,productType,totalInventory,tracksInventory,status,tags,vendor,publishedAt,price_range,created,updated,variants,handle,storefrontId,Reporting_Category - custom,CountryofOrigin - custom,size_value - custom,size_label - custom,uom - custom,far_met_variant_id - custom,amazon_bullet_point_1 - custom,amazon_bullet_point_2 - custom,amazon_bullet_point_3 - custom,amazon_bullet_point_4 - custom,amazon_bullet_point_5 - custom,Createdon - custom,Supplying - custom,catalog_pricing - custom,Ingredients - plytix,Substance - plytix,Packaging_Type - plytix,Deal - plytix,all_natural - plytix,Additive_Free - plytix,Is_Kosher - plytix,Is_Certified_Gluten_Free - plytix,is_Naturally_Gluten_Free - plytix,Is_GMO_Free - plytix,Contains_100_Organic_Ingredients - plytix,Is_Organic - plytix,Is_All_Natural - plytix,Salt_Free - plytix,Is_Vegan - plytix,Is_Keto_Friendly - plytix,Sugar_Conscious - plytix,Seed_oil_Free - plytix,100_Real_Fruit - plytix,Soy_Free - plytix,Sugar_Alternative - plytix,Caffeine_Free - plytix,Free_Range - plytix,Hormone_Free_and_Antibiotic_free - plytix,Extra_Virgin - plytix,Award_Winning - plytix,High_Protein - plytix,High_Fiber - plytix,Iodine_Free - plytix,Low_Sodium - plytix,Gift - plytix,Is_Vegetarian - plytix,Free_of_Artificial_Colorings - plytix,Single_Ingredient - plytix,Superfood - plytix,serving_size_consumer_friendly - nutrition,fibre_daily_value_percent - nutrition,iron - nutrition,calcium - nutrition,protein - nutrition,sugar - nutrition,carbohydrate_daily_value_percent - nutrition,carbohydrate - nutrition,sodium_daily_percentage - nutrition,sodium - nutrition,cholesterol - nutrition,fat_total_daily_percentage - nutrition,fat_total - nutrition,calories - nutrition,vitamin_c - nutrition,vitamin_a - nutrition,trans_fat - nutrition,saturated_fat_daily_percentage - nutrition,saturated_total - nutrition,fibre - nutrition,serving_size - nutrition,BrandDisplayName - custom
0,gid://shopify/Product/9567940280640,Premium Leather Iphone 12,1,"Premium Leather Iphone 12 case. Dimensions: 5.9"" x 3"" x 0.5"" Weight: 0.16 lbs",{'originalSrc': 'https://cdn.shopify.com/s/files/1/0873/2632/7104/files/CAN2120T_F.jpg?v=1727299856'},"{'data': [{'originalSrc': 'https://cdn.shopify.com/s/files/1/0873/2632/7104/files/CAN2120T_F.jpg?v=1727299856'}, {'originalSrc': 'https://cdn.shopify.com/s/files/1/0873/2632/7104/files/CAN2120T_B.jpg?v=1727299857'}, {'originalSrc': 'https://cdn.shopify.com/s/files/1/0873/2632/7104/files/CAN2120T_S1.jpg?v=1727299856'}, {'originalSrc': 'https://cdn.shopify.com/s/files/1/0873/2632/7104/files/CAN2120T_S2.jpg?v=1727299857'}]}",None,NaN,,0,False,ACTIVE,[],Cananu,2021-01-29T13:29:04Z,"{'max': {'amount': '17.71', 'currency': 'CAD'}, 'min': {'amount': '17.71', 'currency': 'CAD'}}",2024-08-28T05:41:55Z,2024-10-01T22:55:12Z,"{'data': [{'id': 'gid://shopify/ProductVariant/49594307641664', 'sku': 'CAN2120T', 'title': 'Default Title', 'displayName': 'Premium Leather Iphone 12 - Default Title', 'inventoryQuantity': 21, 'price': '17.71', 'created': '2024-09-25T00:25:15Z', 'updated': '2024-08-28T05:41:56Z'}]}",premium-leather-iphone-12,gid://shopify/Product/9567940280640,Wares / Accessories / IPhone Cases,China,1.0,Case Teal 1 pc,pc,4.046192e+13,"SLIM DESIGN - This case will fit snugly around your iPhone, allowing your iPhone to rest comfortably in your pocket.",PREMIUM LEATHER - The premium leather we use for our cases will elevate the look of your iPhone while also allowing you to maintain a firm grip when you are holding your device.,"ADDED PROTECTION - Use this leather case, and your phone will be able to withstand bumps or drops.",A GREAT GIFT - This Premium Leather iPhone Case makes the perfect gift for anyone who has recently purchased a new iPhone or is looking to change up the look of their iPhone.,YOUR FAVOURITE COLOUR - We offer Premium Leather iPhone Cases in a variety of colours.,2021-01-29,"Default - Dry Ship, Dry Storage","{""FM Pricelist

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2443 entries, 0 to 2442
Data columns (total 92 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            2443 non-null   object 
 1   title                                         2443 non-null   object 
 2   totalVariants                                 2443 non-null   int64  
 3   description                                   2443 non-null   object 
 4   featuredImage                                 2441 non-null   object 
 5   images                                        2443 non-null   object 
 6   category                                      9 non-null      object 
 7   store_url                                     0 non-null      float64
 8   productType                                   2443 non-null   object 
 9   totalInventory                                2443 non-null   i

In [23]:
df.columns.to_list()

['id',
 'title',
 'totalVariants',
 'description',
 'featuredImage',
 'images',
 'category',
 'store_url',
 'productType',
 'totalInventory',
 'tracksInventory',
 'status',
 'tags',
 'vendor',
 'publishedAt',
 'price_range',
 'created',
 'updated',
 'variants',
 'handle',
 'storefrontId',
 'Reporting_Category - custom',
 'CountryofOrigin - custom',
 'size_value - custom',
 'size_label - custom',
 'uom - custom',
 'far_met_variant_id - custom',
 'amazon_bullet_point_1 - custom',
 'amazon_bullet_point_2 - custom',
 'amazon_bullet_point_3 - custom',
 'amazon_bullet_point_4 - custom',
 'amazon_bullet_point_5 - custom',
 'Createdon - custom',
 'Supplying - custom',
 'catalog_pricing - custom',
 'Ingredients - plytix',
 'Substance - plytix',
 'Packaging_Type - plytix',
 'Deal - plytix',
 'all_natural - plytix',
 'Additive_Free - plytix',
 'Is_Kosher - plytix',
 'Is_Certified_Gluten_Free - plytix',
 'is_Naturally_Gluten_Free - plytix',
 'Is_GMO_Free - plytix',
 'Contains_100_Organic_Ingredien

### Selecting Required Columns

In [24]:
required_columns=['id',
 'title',
 'description',
 'featuredImage',
 'images',
 'productType',
 'tags',
 'vendor',
 'price_range',
 'variants',
 'Reporting_Category - custom',
 'CountryofOrigin - custom',
 'amazon_bullet_point_1 - custom',
 'amazon_bullet_point_2 - custom',
 'amazon_bullet_point_3 - custom',
 'amazon_bullet_point_4 - custom',
 'amazon_bullet_point_5 - custom',
 'Ingredients - plytix',
 'Substance - plytix',
 'Packaging_Type - plytix',
 'Deal - plytix',
 'all_natural - plytix',
 'Additive_Free - plytix',
 'Is_Kosher - plytix',
 'Is_Certified_Gluten_Free - plytix',
 'is_Naturally_Gluten_Free - plytix',
 'Is_GMO_Free - plytix',
 'Contains_100_Organic_Ingredients - plytix',
 'Is_Organic - plytix',
 'Is_All_Natural - plytix',
 'Salt_Free - plytix',
 'Is_Vegan - plytix',
 'Is_Keto_Friendly - plytix',
 'Sugar_Conscious - plytix',
 'Seed_oil_Free - plytix',
 '100_Real_Fruit - plytix',
 'Soy_Free - plytix',
 'Sugar_Alternative - plytix',
 'Caffeine_Free - plytix',
 'Free_Range - plytix',
 'Hormone_Free_and_Antibiotic_free - plytix',
 'Extra_Virgin - plytix',
 'Award_Winning - plytix',
 'High_Protein - plytix',
 'High_Fiber - plytix',
 'Iodine_Free - plytix',
 'Low_Sodium - plytix',
 'Is_Vegetarian - plytix',
 'Free_of_Artificial_Colorings - plytix',
 'Single_Ingredient - plytix',
 'Superfood - plytix',
 'iron - nutrition',
 'calcium - nutrition',
 'protein - nutrition',
 'sugar - nutrition',
 'carbohydrate - nutrition',
 'sodium - nutrition',
 'cholesterol - nutrition',
 'fat_total - nutrition',
 'calories - nutrition',
 'vitamin_c - nutrition',
 'vitamin_a - nutrition',
 'trans_fat - nutrition',
 'saturated_total - nutrition',
 'fibre - nutrition',
 'serving_size - nutrition',
 'BrandDisplayName - custom']

formatted_df = df[required_columns]
formatted_df.head(5)

,id,title,description,featuredImage,images,productType,tags,vendor,price_range,variants,Reporting_Category - custom,CountryofOrigin - custom,amazon_bullet_point_1 - custom,amazon_bullet_point_2 - custom,amazon_bullet_point_3 - custom,amazon_bullet_point_4 - custom,amazon_bullet_point_5 - custom,Ingredients - plytix,Substance - plytix,Packaging_Type - plytix,Deal - plytix,all_natural - plytix,Additive_Free - plytix,Is_Kosher - plytix,Is_Certified_Gluten_Free - plytix,is_Naturally_Gluten_Free - plytix,Is_GMO_Free - plytix,Contains_100_Organic_Ingredients - plytix,Is_Organic - plytix,Is_All_Natural - plytix,Salt_Free - plytix,Is_Vegan - plytix,Is_Keto_Friendly - plytix,Sugar_Conscious - plytix,Seed_oil_Free - plytix,100_Real_Fruit - plytix,Soy_Free - plytix,Sugar_Alternative - plytix,Caffeine_Free - plytix,Free_Range - plytix,Hormone_Free_and_Antibiotic_free - plytix,Extra_Virgin - plytix,Award_Winning - plytix,High_Protein - plytix,High_Fiber - plytix,Iodine_Free - plytix,Low_Sodium - plytix,Is_Vegetarian - plytix,Free_of_Artificial_Colorings - plytix,Single_Ingredient - plytix,Superfood - plytix,iron - nutrition,calcium - nutrition,protein - nutrition,sugar - nutrition,carbohydrate - nutrition,sodium - nutrition,cholesterol - nutrition,fat_total - nutrition,calories - nutrition,vitamin_c - nutrition,vitamin_a - nutrition,trans_fat - nutrition,saturated_total - nutrition,fibre - nutrition,serving_size - nutrition,BrandDisplayName - custom
0,gid://shopify/Product/9567940280640,Premium Leather Iphone 12,"Premium Leather Iphone 12 case. Dimensions: 5.9"" x 3"" x 0.5"" Weight: 0.16 lbs",{'originalSrc': 'https://cdn.shopify.com/s/files/1/0873/2632/7104/files/CAN2120T_F.jpg?v=1727299856'},"{'data': [{'originalSrc': 'https://cdn.shopify.com/s/files/1/0873/2632/7104/files/CAN2120T_F.jpg?v=1727299856'}, {'originalSrc': 'https://cdn.shopify.com/s/files/1/0873/2632/7104/files/CAN2120T_B.jpg?v=1727299857'}, {'originalSrc': 'https://cdn.shopify.com/s/files/1/0873/2632/7104/files/CAN2120T_S1.jpg?v=1727299856'}, {'originalSrc': 'https://cdn.shopify.com/s/files/1/0873/2632/7104/files/CAN2120T_S2.jpg?v=1727299857'}]}",,[],Cananu,"{'max': {'amount': '17.71', 'currency': 'CAD'}, 'min': {'amount': '17.71', 'currency': 'CAD'}}","{'data': [{'id': 'gid://shopify/ProductVariant/49594307641664', 'sku': 'CAN2120T', 'title': 'Default Title', 'displayName': 'Premium Leather Iphone 12 - Default Title', 'inventoryQuantity': 21, 'price': '17.71', 'created': '2024-09-25T00:25:15Z', 'updated': '2024-08-28T05:41:56Z'}]}",Wares / Accessories / IPhone Cases,China,"SLIM DESIGN - This case will fit snugly around your iPhone, allowing your iPhone to rest comfortably in your pocket.",PREMIUM LEATHER - The premium leather we use for our cases will elevate the look of your iPhone while also allowing you to maintain a firm grip when you are holding your device.,"ADDED PROTECTION - Use this leather case, and your phone will be able to withstand bumps or drops.",A GREAT GIFT - This Premium Leather iPhone Case makes the perfect gift for anyone who has recently purchased a new iPhone or is looking to change up the look of their iPhone.,YOUR FAVOURITE COLOUR - We offer Premium Leather iPhone Cases in a variety of colours.,leather,whole large item,9.0,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,gid://shopify/Product/9567940346176,Premium Leather Iphone 12,"Premium Leather Iphone 12 Pro case. Dimensions: 5.9"" x 3"" x 0.5"" Weight: 0.16 lbs",{'originalSrc': 'https://cdn.shopify.com/s/files/1/0873/2632/7104/files/CAN2120R_B.jpg?v=1727299867'},"{'data': [{'originalSrc': 'https://cdn.shopify.com/s/files/1/0873/2632/7104/files/CAN2120R_B.jpg?v=1727299867'}, {'originalSrc': 'https://cdn.shopify.com/s/files/1/0873/2632/7104/files/CAN2120R_S2.jpg?v=1727299868'}]}",,[],Cananu,"{'ma

In [25]:
formatted_df.columns

Index(['id', 'title', 'description', 'featuredImage', 'images', 'productType', 'tags', 'vendor', 'price_range', 'variants', 'Reporting_Category - custom', 'CountryofOrigin - custom', 'amazon_bullet_point_1 - custom', 'amazon_bullet_point_2 - custom', 'amazon_bullet_point_3 - custom', 'amazon_bullet_point_4 - custom', 'amazon_bullet_point_5 - custom', 'Ingredients - plytix', 'Substance - plytix', 'Packaging_Type - plytix', 'Deal - plytix', 'all_natural - plytix', 'Additive_Free - plytix', 'Is_Kosher - plytix', 'Is_Certified_Gluten_Free - plytix', 'is_Naturally_Gluten_Free - plytix', 'Is_GMO_Free - plytix', 'Contains_100_Organic_Ingredients - plytix', 'Is_Organic - plytix', 'Is_All_Natural - plytix', 'Salt_Free - plytix', 'Is_Vegan - plytix', 'Is_Keto_Friendly - plytix', 'Sugar_Conscious - plytix', 'Seed_oil_Free - plytix', '100_Real_Fruit - plytix', 'Soy_Free - plytix', 'Sugar_Alternative - plytix', 'Caffeine_Free - plytix', 'Free_Range - plytix',
       'Hormone_Free_and_Antibiotic_fre

### Formatting Column Names

In [26]:
#removing (.*) data from column names
replacer = lambda x: re.sub(r'\s{2,}','',re.sub(r'\((.*)\)', '', x).strip()).strip().replace("nutrition","").replace("custom","").replace("plytix","").replace(" - ",' ').strip().replace(" ",'_')
formatted_df.rename(replacer, axis='columns', inplace=True)
formatted_df.rename(columns={
    "amazon_bullet_point_1":"bullet_point_1",
    "amazon_bullet_point_2":"bullet_point_2",
    "amazon_bullet_point_3":"bullet_point_3",
    "amazon_bullet_point_4":"bullet_point_4",
    "amazon_bullet_point_5":"bullet_point_5",
    "productType":"product_type",
    "featuredImage":"featured_image",
    "CountryofOrigin":"country_of_origin",
    "BrandDisplayName":"brand_display_name",
},inplace=True)
formatted_df.rename(str.lower, axis='columns', inplace=True)
formatted_df.columns.to_list()

/tmp/ipykernel_176844/1039692606.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  formatted_df.rename(replacer, axis='columns', inplace=True)
/tmp/ipykernel_176844/1039692606.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  formatted_df.rename(columns={
/tmp/ipykernel_176844/1039692606.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  formatted_df.rename(str.lower, axis='columns', inplace=True)


['id',
 'title',
 'description',
 'featured_image',
 'images',
 'product_type',
 'tags',
 'vendor',
 'price_range',
 'variants',
 'reporting_category',
 'country_of_origin',
 'bullet_point_1',
 'bullet_point_2',
 'bullet_point_3',
 'bullet_point_4',
 'bullet_point_5',
 'ingredients',
 'substance',
 'packaging_type',
 'deal',
 'all_natural',
 'additive_free',
 'is_kosher',
 'is_certified_gluten_free',
 'is_naturally_gluten_free',
 'is_gmo_free',
 'contains_100_organic_ingredients',
 'is_organic',
 'is_all_natural',
 'salt_free',
 'is_vegan',
 'is_keto_friendly',
 'sugar_conscious',
 'seed_oil_free',
 '100_real_fruit',
 'soy_free',
 'sugar_alternative',
 'caffeine_free',
 'free_range',
 'hormone_free_and_antibiotic_free',
 'extra_virgin',
 'award_winning',
 'high_protein',
 'high_fiber',
 'iodine_free',
 'low_sodium',
 'is_vegetarian',
 'free_of_artificial_colorings',
 'single_ingredient',
 'superfood',
 'iron',
 'calcium',
 'protein',
 'sugar',
 'carbohydrate',
 'sodium',
 'cholesterol'

In [27]:
formatted_df.dtypes

id                                   object
title                                object
description                          object
featured_image                       object
images                               object
product_type                         object
tags                                 object
vendor                               object
price_range                          object
variants                             object
reporting_category                   object
country_of_origin                    object
bullet_point_1                       object
bullet_point_2                       object
bullet_point_3                       object
bullet_point_4                       object
bullet_point_5                       object
ingredients                          object
substance                            object
packaging_type                      float64
deal                                 object
all_natural                          object
additive_free                   

### Creating json file for shopify products to use later

In [28]:
formatted_df.to_json("datas/shopify_products.json", orient="records", indent=4)


In [29]:
formatted_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2443 entries, 0 to 2442
Data columns (total 67 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                2443 non-null   object 
 1   title                             2443 non-null   object 
 2   description                       2443 non-null   object 
 3   featured_image                    2441 non-null   object 
 4   images                            2443 non-null   object 
 5   product_type                      2443 non-null   object 
 6   tags                              2443 non-null   object 
 7   vendor                            2443 non-null   object 
 8   price_range                       2443 non-null   object 
 9   variants                          2443 non-null   object 
 10  reporting_category                2336 non-null   object 
 11  country_of_origin                 2291 non-null   object 
 12  bullet

In [30]:
formatted_df.loc[:,'custom_type']=formatted_df.loc[:,'reporting_category'].apply(lambda x: str(x).split("/")[0])
formatted_df.tail()

/tmp/ipykernel_176844/25400920.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  formatted_df.loc[:,'custom_type']=formatted_df.loc[:,'reporting_category'].apply(lambda x: str(x).split("/")[0])


,id,title,description,featured_image,images,product_type,tags,vendor,price_range,variants,reporting_category,country_of_origin,bullet_point_1,bullet_point_2,bullet_point_3,bullet_point_4,bullet_point_5,ingredients,substance,packaging_type,deal,all_natural,additive_free,is_kosher,is_certified_gluten_free,is_naturally_gluten_free,is_gmo_free,contains_100_organic_ingredients,is_organic,is_all_natural,salt_free,is_vegan,is_keto_friendly,sugar_conscious,seed_oil_free,100_real_fruit,soy_free,sugar_alternative,caffeine_free,free_range,hormone_free_and_antibiotic_free,extra_virgin,award_winning,high_protein,high_fiber,iodine_free,low_sodium,is_vegetarian,free_of_artificial_colorings,single_ingredient,superfood,iron,calcium,protein,sugar,carbohydrate,sodium,cholesterol,fat_total,calories,vitamin_c,vitamin_a,trans_fat,saturated_total,fibre,serving_size,brand_display_name,custom_type
2438,gid://shopify/Product/9601329561920,Sunflower Oil Infused with Fir Tree Strong & Woody Soliam Organic,"Sunflower Oil Infused with Fir Tree is a gastronomic sunflower-based oil made by Abies Lagrimus and obtained from a several-weeks cold maceration with fir tree cones in order to extract the best aromas and its unique taste. With its subtle golden color, this oil has woody and resinous aromas, powerful on the palate, mixed with surprising smells of forest and undergrowth.",{'originalSrc': 'https://cdn.shopify.com/s/files/1/0873/2632/7104/files/131861_F.jpg?v=1727239737'},"{'data': [{'originalSrc': 'https://cdn.shopify.com/s/files/1/0873/2632/7104/files/131861_F.jpg?v=1727239737'}, {'originalSrc': 'https://cdn.shopify.com/s/files/1/0873/2632/7104/files/131861_B.jpg?v=1727239736'}]}",,[],Abies Lagrimuss,"{'max': {'amount': '41.29', 'currency': 'CAD'}, 'min': {'amount': '41.29', 'currency': 'CAD'}}","{'data': [{'id': 'gid://shopify/ProductVariant/49729255964992', 'sku': '131861', 'title': 'Default Title', 'displayName': 'Sunflower Oil Infused with Fir Tree Strong & Woody Soliam Organic - Default Title', 'inventoryQuantity': 0, 'price': '41.29', 'created': '2024-09-26T05:14:21Z', 'updated': '2024-09-25T03:30:52Z'}]}","Oils, Olives & Vinegars / Nut & Seed Oils",France,Infused with robust fir tree essence for a bold flavor profile,"Ideal for adding depth to grilled meats, roasted potatoes, and hearty stews",Crafted with organic ingredients for an authentic culinary experience,Elevate your dishes with a bold infusion of sunflower oil and fir tree essence,"Convenient 250 ml bottle, perfect for home kitchens",Sunflower Oil infused with fir tree cones.,Liquid Oil,8.0,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Oils, Olives & Vinegars"
2439,gid://shopify/Product/9601346699584,White Anchovy Fillets,"Marinated fresh white anchovies prepared in this Spanish style are called ""Boquarones"". A Mediterranean delicacy; marinated white anchovies can be enjoyed straight from the package showcasing their mild fish flavour and lovely white fillet. Spur of the moment entertaining? You will always be ready to serve superb tapas of roasted sweet red peppers topped with white anchovy fillets on a crostini with a splash of vinegar. Mario Batali is a huge fan of these white anchovies. Try with a glass of dry sherry and you will not be dissapointed.",{'originalSrc': 'https://cdn.shopify.com/s/files/1/0873/2632/7104/files/091075_F.jpg?v=1727239783'},"{'data': [{'originalSrc': 'https://cdn.shopify.com/s/files/1/0873/2632/7104/files/091075_F.jpg?v=1727239783'}, {'originalSrc': 'https://cdn.shopify.com/s/files/1/0873/2632/7104/files/091075_B.jpg?v=1727239783'}]}",,[],Agostino Recca,"{'max': {'amount': '22.41', 'currency': 'CAD'}, 'min': {'amount': '22.41', 'currency': 'CAD'}}","{'data': [{'id': 'gid://shopify/ProductVariant/49729319043392', 'sku': '091075', 'title': 'Default Title', 

# FIELDS TO USE FOR RECOMMENDATION

* TITLE
* DESCRIPTION
* BULLET_POINT_1
* BULLET_POINT_2
* BULLET_POINT_3
* BULLET_POINT_4
* BULLET_POINT_5
* PRODUCT_TYPE
* SUBSTANCE
* COUNTRY_OF_ORIGIN
* BRAND_DISPLAY_NAME
* CUSTOM_TYPE
* ADDICTIVE_FREE
* IS_KOSHER
* IS_CERTIFIED_GLUTEN_FREE
* IS_NATURALLY_GLUTEN_FREE
* IS_GMO_FREE
* CONTAINS_100_ORGANIC_INGREDIENTS
* IS_ORGANIC
* IS_ALL_NATURAL
* SALT_FREE
* IS_VEGAN
* SUGAR_CONSCIOUS
* SEED_OIL_FREE
* 100_REAL_FRUIT
* SOY_FREE
* SUGAR_ALTERNATIVE
* Caffeine_Free
* Free_Range
* Hormone_Free_And_Antibiotic_Free
* Extra_Virgin
* Award_Winning
* High_Protein
* High_Fiber
* Low_Sodium
* Is_Vegetarian
* Free_of_Artificial_Colorings
* Single_Ingredient
* Superfood

## QUANITY TO USE FOR RECOMMENDATION
* IRON
* CALCIUM
* PROTEIN
* SUGAR
* CARBOHYDRATE
* SODIUM
* CHOLESTEROL
* FAT_TOTAL
* CALORIES
* VITAMIN_C
* VITAMIN_A
* TRANS_FAT
* SATURATED_TOTAL
* FIBRE



### Creating embedding content for recommendation

* This usages previously created json file for context creation. So we can use it in the application too.

In [35]:
def column_handler(condition, value):
    match condition:
        case "title":
            return f"{value} " if value else ""
        case "description":
            return f"{value} " if value else ""
        case "product_type":
            return f"Product type: {value}. " if value else ""
        case "tags":
            if isinstance(value, str) and str(value).startswith("[") and str(value).endswith("]"):
                tags = eval(value)
            else:
                tags = value
            if isinstance(tags, list):
                return "tags:" + ", ".join(tags) + ". " if len(tags) > 0 else ""
            else:
                return f"tags: {value}. formatted_df.loc[:'sparse_vector'] = " if value else ""
        case "vendor":
            return f"vendor: {value}. " if value else ""
        case "price_range":
            if isinstance(value, str) and str(value).startswith("{") and str(value).endswith("}"):

                prices = eval(value)
            else:
                prices = value
            max = prices.get('max',{}).get('amount',0)
            min = prices.get('min',{}).get('amount',0)
            currency = prices.get('max',{}).get('currency',"CAD")
            if max == min:
                return f"Product price: {max} {currency}. "
            else:
                return f"Product price: {min} - {max} {currency}. "
        case "reporting_category":
            return f"category: {value.replace('/',',')}. " if value else ""
        case "country_of_origin":
            return f"created in: {value}. " if value else ""
        case "bullet_point_1":
            return f"{value} " if value else ""
        case "bullet_point_2":
            return f"{value} " if value else ""
        case "bullet_point_3":
            return f"{value} " if value else ""
        case "bullet_point_4":
            return f"{value} " if value else ""
        case "bullet_point_5":
            return f"{value} " if value else ""
        case "ingredients":
            return f"Ingredients: {value}. " if value else ""
        case "substance":
            return f"Substance: {value}. " if value else ""
        
        # Boolean columns here
        case "all_natural":
            return "All natural. " if value and value=="true" else ""
        case "additive_free":
            return "Additive free. " if value and value=="true" else ""
        case "is_kosher":
            return "Kosher. " if value and value=="true" and value=="true" else ""
        case "is_certified_gluten_free":
            return "Gluten free. " if value and value=="true" else ""
        case "is_naturally_gluten_free":
            return "Gluten free. " if value and value=="true" else ""
        case "is_gmo_free":
            return "GMO free. " if value and value=="true" else ""
        case "contains_100_organic_ingredients":
            return "100 Organic ingredients. " if value and value=="true" else ""
        case "is_organic":
            return "Organic. " if value and value=="true" else ""
        case "is_all_natural":
            return "All natural. " if value and value=="true" else ""
        case "salt_free":
            return "Salt free. " if value and value=="true" else ""
        case "is_vegan":
            return "Vegan. " if value and value=="true" else ""
        case "is_keto_friendly":
            return "Keto friendly. " if value and value=="true" else ""
        case "sugar":
            return "Sugar free. " if value and value=="true" else ""
        case "caffeine_free":
            return "Caffeine free. " if value and value=="true" else ""
        case "high_protein":
            return "High in protein. " if value and value=="true" else ""
        case "high_fiber":
            return "High in fiber. " if value and value=="true" else ""
        case "low_sodium":
            return "Low in sodium. " if value and value=="true" else ""
        case "superfood":
            return "Superfood. " if value and value=="true" else ""
        case "is_vegetarian":
            return "Vegetarian. " if value and value=="true" else ""
        case "free_of_artificial_colorings":
            return "Artificial Color free. " if value and value=="true" else ""
        
        # Numeric columns here
        case "calories":
            return f"{value} calories. " if value else ""
        case "carbohydrate":
            return f"{value} carbs. " if value else ""
        case "protein":
            return f"{value} proteins. " if value else ""
        case "calcium":
            return f"{value} Calcium. " if value else ""
        case "vitamin_c":
            return f"{value} Vitamin C. " if value else ""
        case "vitamin_a":
            return f"{value} Vitamin A. " if value else ""
        case "trans_fat":
            return f"{value} Trans fat. " if value else ""
        case "fat_total":
            return f"{value} fat. " if value else ""
        case "fibre":
            return f"{value} Fibre. " if value else ""
        case "iron":
            return f"{value} Iron. " if value else ""
        case _:
            return ""

## Function To create context

In [36]:
with open('datas/shopify_products.json','r') as f:
    formatted_shopify_products = json.load(f)
shopify_product_map = { prod['id']:prod for prod in formatted_shopify_products}

def create_embedding_context(id, map_data=shopify_product_map):
    embedding = ""
    data = map_data[id]
    for key, value in data.items():
        concat = column_handler(key, value)
        embedding = embedding + concat
    return embedding

### Applying function

In [37]:
formatted_df.loc[:,'context']=formatted_df.loc[:,'id'].apply(create_embedding_context)

In [38]:
formatted_df.tail()

,id,title,description,featured_image,images,product_type,tags,vendor,price_range,variants,reporting_category,country_of_origin,bullet_point_1,bullet_point_2,bullet_point_3,bullet_point_4,bullet_point_5,ingredients,substance,packaging_type,deal,all_natural,additive_free,is_kosher,is_certified_gluten_free,is_naturally_gluten_free,is_gmo_free,contains_100_organic_ingredients,is_organic,is_all_natural,salt_free,is_vegan,is_keto_friendly,sugar_conscious,seed_oil_free,100_real_fruit,soy_free,sugar_alternative,caffeine_free,free_range,hormone_free_and_antibiotic_free,extra_virgin,award_winning,high_protein,high_fiber,iodine_free,low_sodium,is_vegetarian,free_of_artificial_colorings,single_ingredient,superfood,iron,calcium,protein,sugar,carbohydrate,sodium,cholesterol,fat_total,calories,vitamin_c,vitamin_a,trans_fat,saturated_total,fibre,serving_size,brand_display_name,custom_type,context
2438,gid://shopify/Product/9601329561920,Sunflower Oil Infused with Fir Tree Strong & Woody Soliam Organic,"Sunflower Oil Infused with Fir Tree is a gastronomic sunflower-based oil made by Abies Lagrimus and obtained from a several-weeks cold maceration with fir tree cones in order to extract the best aromas and its unique taste. With its subtle golden color, this oil has woody and resinous aromas, powerful on the palate, mixed with surprising smells of forest and undergrowth.",{'originalSrc': 'https://cdn.shopify.com/s/files/1/0873/2632/7104/files/131861_F.jpg?v=1727239737'},"{'data': [{'originalSrc': 'https://cdn.shopify.com/s/files/1/0873/2632/7104/files/131861_F.jpg?v=1727239737'}, {'originalSrc': 'https://cdn.shopify.com/s/files/1/0873/2632/7104/files/131861_B.jpg?v=1727239736'}]}",,[],Abies Lagrimuss,"{'max': {'amount': '41.29', 'currency': 'CAD'}, 'min': {'amount': '41.29', 'currency': 'CAD'}}","{'data': [{'id': 'gid://shopify/ProductVariant/49729255964992', 'sku': '131861', 'title': 'Default Title', 'displayName': 'Sunflower Oil Infused with Fir Tree Strong & Woody Soliam Organic - Default Title', 'inventoryQuantity': 0, 'price': '41.29', 'created': '2024-09-26T05:14:21Z', 'updated': '2024-09-25T03:30:52Z'}]}","Oils, Olives & Vinegars / Nut & Seed Oils",France,Infused with robust fir tree essence for a bold flavor profile,"Ideal for adding depth to grilled meats, roasted potatoes, and hearty stews",Crafted with organic ingredients for an authentic culinary experience,Elevate your dishes with a bold infusion of sunflower oil and fir tree essence,"Convenient 250 ml bottle, perfect for home kitchens",Sunflower Oil infused with fir tree cones.,Liquid Oil,8.0,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Oils, Olives & Vinegars","Sunflower Oil Infused with Fir Tree Strong & Woody Soliam Organic Sunflower Oil Infused with Fir Tree is a gastronomic sunflower-based oil made by Abies Lagrimus and obtained from a several-weeks cold maceration with fir tree cones in order to extract the best aromas and its unique taste. With its subtle golden color, this oil has woody and resinous aromas, powerful on the palate, mixed with surprising smells of forest and undergrowth. vendor: Abies Lagrimuss. Product price: 41.29 CAD. category: Oils, Olives & Vinegars , Nut & Seed Oils. created in: France. Infused with robust fir tree essence for a bold flavor profile Ideal for adding depth to grilled meats, roasted potatoes, and hearty stews Crafted with organic ingredients for an authentic culinary experience Elevate your dishes with a bold infusion of sunflower oil and fir tree essence Convenient 250 ml bottle, perfect for home kitchens Ingredients: Sunflower Oil infused with fir tree cones.. Substance: Liquid Oil. Organic."
2439,gid://shopify/Product/9601346699584,White Anchovy Fillets,"Marinated fresh white anchovies prepared in this Spanish style are called ""Boqua

In [50]:
import tabulate

table = tabulate.tabulate(
    formatted_df[['id','title','description','context']].tail(10),
    headers="keys",
    tablefmt="simple_grid",
    missingval="---------------------"
)
print(table)

┌──────┬─────────────────────────────────────┬───────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────